--- Day 11: Hex Ed ---

Crossing the bridge, you've barely reached the other side of the stream when a program comes up to you, clearly in distress. "It's my child process," she says, "he's gotten lost in an infinite grid!"

Fortunately for her, you have plenty of experience with infinite grids.

Unfortunately for you, it's a hex grid.

The hexagons ("hexes") in this grid are aligned such that adjacent hexes can be found to the north, northeast, southeast, south, southwest, and northwest:

```
  \ n  /
nw +--+ ne
  /    \
-+      +-
  \    /
sw +--+ se
  / s  \
```

You have the path the child process took. Starting where he started, you need to determine the fewest number of steps required to reach him. (A "step" means to move from the hex you are in to any adjacent hex.)

For example:

ne,ne,ne is 3 steps away.
ne,ne,sw,sw is 0 steps away (back where you started).
ne,ne,s,s is 2 steps away (se,se).
se,sw,se,sw,sw is 3 steps away (s,s,sw).

In [1]:
start = [0, 0]

In [2]:
with open("day11.txt") as f:
    data = f.readlines()

In [3]:
steps = [x.split(",") for x in data][0]

In [4]:
steps[0:10]

['n', 'nw', 'nw', 'nw', 'sw', 'sw', 'sw', 'ne', 's', 's']

In [5]:
len(steps)

8223

In [6]:
n = [0, 1.0]
ne = [1, 0.5]
se = [1, -0.5]
s = [0, -1.0]
sw = [-1, -0.5]
nw = [-1, 0.5]

In [8]:
# test cases
# ne,ne,ne = 3
test1 = start[0] + ne[0], start[1] + ne[1]
test1 = test1[0] + ne[0], test1[1] + ne[1]
test1 = test1[0] + ne[0], test1[1] + ne[1]
test1

(3, 1.5)

In [9]:
def move(start, step):
    new_loc = start[0] + step[0], start[1] + step[1]
    return new_loc

In [10]:
# ne,ne,sw,sw = 0
test2_steps = [ne,ne,sw,sw]
test2 = start
for x in test2_steps:
    test2 = move(test2, x)
test2

(0, 0.0)

In [11]:
# ne,ne,s,s = 2
test3 = start[0] + ne[0], start[1] + ne[1]
test3 = test3[0] + ne[0], test3[1] + ne[1]
test3 = test3[0] + s[0], test3[1] + s[1]
test3 = test3[0] + s[0], test3[1] + s[1]
print(test3)
test3_steps = [ne,ne,s,s]
test3 = start
for x in test3_steps:
    test3 = move(test3, x)
print(test3)

(2, -1.0)
(2, -1.0)


In [12]:
# se,sw,se,sw,sw = 3
test4_steps = [se,sw,se,sw,sw]
test4 = start
for x in test4_steps:
    test4 = move(test4, x)
print(test4)
test4 = move(start, se)
test4 = move(test4, sw)
test4 = move(test4, se)
test4 = move(test4, sw)
test4 = move(test4, sw)
print(test4)

(-1, -2.5)
(-1, -2.5)


In [13]:
for x in test4_steps:
    print(x)

[1, -0.5]
[-1, -0.5]
[1, -0.5]
[-1, -0.5]
[-1, -0.5]


In [14]:
eval(steps[0])

[0, 1.0]

In [33]:
final = start
for x in steps:
    final = move(final, eval(x))
final

(705, -49.5)

In [46]:
def distance_back(coords):
    if abs(coords[0]) >= abs(coords[1]):
        return coords[0]
    # figure out which quadrant to move to center
    if coords[0] >= 0:
        if coords[1] >= 0:
            direction = sw
        if coords[1] < 0:
            direction = nw
    if coords[0] < 0:
        if coords[1] >= 0:
            direction = se
        if coords[1] < 0:
            direction = ne
    backwards = coords
    countdown = abs(coords[0])
    while countdown > 0:
        backwards = move(backwards, direction)
        countdown += -1
    return abs(coords[0]) + abs(backwards[1])

In [47]:
distance_back(final)

705

That's the right answer! You are one gold star closer to debugging the printer. 

--- Part Two ---

How many steps away is the furthest he ever got from his starting position?


In [48]:
pt2_steps = {}
pt2 = start
counter = 1
for x in steps:
    pt2 = move(pt2, eval(x))
    pt2_steps[counter] = pt2
    counter += 1

In [49]:
counter

8224

In [50]:
import pandas as pd

In [51]:
df = pd.DataFrame.from_dict(pt2_steps, orient="index")
df.head()

,0,1
1,0,1.0
2,-1,1.5
3,-2,2.0
4,-3,2.5
5,-4,2.0


In [52]:
df.rename({0: 'x', 1: 'y'}, axis='columns', inplace=True)

In [53]:
df.describe()

,x,y
count,8223.000000,8223.000000
mean,566.692205,-74.344765
std,630.360153,316.193147
min,-412.000000,-629.500000
25%,-31.500000,-321.000000
50%,748.000000,-54.000000
75%,1130.000000,183.500000
max,1469.000000,466.000000


In [54]:
max_x = df[df['x'] == max(df['x'])]
df_max_x = max_x[max_x['y'] == max(max_x['y'])]
df_max_x

,x,y
6798,1469,265.5


In [61]:
min_x = df[df['x'] == min(df['x'])]
df_min_x = min_x[min_x['y'] == min(min_x['y'])]
df_min_x

,x,y
1936,-412,-124.0


In [55]:
max_y = df[df['y'] == max(df['y'])]
df_max_y = max_y[max_y['x'] == max(max_y['x'])]
df_max_y

,x,y
6309,1242,466.0


In [62]:
min_y = df[df['y'] == min(df['y'])]
df_min_y = min_y[min_y['x'] == min(min_y['x'])]
df_min_y

,x,y
3419,321,-629.5


In [63]:
max_x_coor = df_max_x.x.values.tolist() + df_max_x.y.values.tolist()
min_x_coor = df_min_x.x.values.tolist() + df_min_x.y.values.tolist()
max_y_coor = df_max_y.x.values.tolist() + df_max_y.y.values.tolist()
min_y_coor = df_min_y.x.values.tolist() + df_min_y.y.values.tolist()

In [65]:
coord_opts = [max_x_coor, min_x_coor, max_y_coor, min_y_coor]

In [66]:
for coord in coord_opts:
    print(coord, distance_back(coord))

[1469, 265.5] 1469
[-412, -124.0] -412
[1242, 466.0] 1242
[321, -629.5] 790.0


That's the right answer! You are one gold star closer to debugging the printer.